In [ ]:
#Connecting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing Required Libries
import os
import zipfile
import tensorflow as tf
import numpy as np
from keras. preprocessing.image import ImageDataGenerator
from keras.models import load_model,save_model
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.applications.resnet import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.applications.densenet import DenseNet121
from keras.optimizers import Adam
from keras.applications.nasnet import NASNetMobile
from tensorflow import keras
from keras.applications import EfficientNetB0


In [ ]:
#Credential of kaggle
#steps: go to kaggle profile > click account > scroll down and click on create new api token and download add the username and kaggle key from downloaded api key token 
#os.environ['KAGGLE_USERNAME'] = "dhanushsuvarna" # username from the json file
#os.environ['KAGGLE_KEY'] = "708fcda2c3a62199540b712b530bdd31" # key from the json file

In [ ]:
#Downloading dataset from kaggle
#go to data and copy the link and paste it in below code
#!kaggle datasets download -d puneet6060/intel-image-classification

 97% 337M/346M [00:07<00:00, 73.4MB/s]
100% 346M/346M [00:08<00:00, 44.9MB/s]


In [ ]:
#unzip the file by providing the zip file location and extracting location
#zip_ref = zipfile.ZipFile("/content/intel-image-classification.zip", 'r')
#zip_ref.extractall("/content/drive/MyDrive/iic2")
#zip_ref.close()


In [ ]:
#data Path of train test and prediction directory
train_data_dir = '/content/drive/MyDrive/iic2/seg_train/seg_train'
test_data_dir =  '/content/drive/MyDrive/iic2/seg_test/seg_test'
pred_dir = '/content/drive/MyDrive/iic2/seg_pred/seg_pred'
inferance_data_dir ='/content/drive/MyDrive/iic2/Inference Data(DX)-20221018T042634Z-001/Inference Data(DX)'

In [ ]:
#Image data genrator for augmentation
IMG_SIZE = (224, 224)
BATCH_SIZE = 100
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)    
# set the directory for train test and validation
train_data = image_generator.flow_from_directory( directory=train_data_dir,
                                                 target_size=IMG_SIZE, 
                                                 subset="training",
                                                 class_mode='categorical',batch_size=BATCH_SIZE)

val_data = image_generator.flow_from_directory(directory=train_data_dir,
                                                 target_size=IMG_SIZE, 
                                                 subset="validation",
                                                 class_mode='categorical',batch_size=BATCH_SIZE)
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(test_data_dir,
                                       target_size=IMG_SIZE, batch_size=BATCH_SIZE,shuffle=False)




Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
#Train and test steps
train_steps = len(train_data)
val_steps  = len(val_data)

In [ ]:
#model creation
#include_top=Flase will allow us to add dense layer at the end
#trainable=Flase will not train the model because the model is pre trained
mobilenet = Sequential()

mobilenet_pretrained_model2= tf.keras.applications.MobileNet(include_top=False,
                   input_shape=(224,224,3),
                   weights='imagenet',dropout=0.4)
for layer in mobilenet_pretrained_model.layers:
        layer.trainable=False

mobilenet.add(mobilenet_pretrained_model)


17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
#flattening the output(coverting two dimentional into one diementional)
mobilenet.add(Flatten())
mobilenet.add(Dense(6, activation='softmax'))

In [ ]:
#Compiling the model
mobilenet.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#model checkpoint will save the weight for every epoch
#early stopping will stop the epoch if the accuracy is not increasing and if we specify the patience it will wait till 6 run if the accuracy is not increasing
mobilenet_checkpoint_filepath2='/content/drive/MyDrive/iic-final/mnet2_my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

mobilenet_call_back2=[ModelCheckpoint(
    filepath=mnet_checkpoint_filepath2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True),EarlyStopping(monitor="val_accuracy",patience=6)]

In [ ]:
#training the model using fit
mobilenet_hist = mobilenet.fit(train_data,validation_data= val_data,epochs=100,callbacks=mnet_call_back2,steps_per_epoch = train_steps,validation_steps=val_steps)


Epoch 1/100
113/113 [==============================] - 2423s 21s/step - loss: 0.4836 - accuracy: 0.8467 - val_loss: 0.3303 - val_accuracy: 0.8969
Epoch 2/100
113/113 [==============================] - 37s 327ms/step - loss: 0.1396 - accuracy: 0.9511 - val_loss: 0.3151 - val_accuracy: 0.9026
Epoch 3/100
113/113 [==============================] - 37s 328ms/step - loss: 0.0605 - accuracy: 0.9836 - val_loss: 0.3036 - val_accuracy: 0.9083
Epoch 4/100
113/113 [==============================] - 37s 326ms/step - loss: 0.0308 - accuracy: 0.9953 - val_loss: 0.3002 - val_accuracy: 0.9112
Epoch 5/100
113/113 [==============================] - 36s 321ms/step - loss: 0.0200 - accuracy: 0.9980 - val_loss: 0.3001 - val_accuracy: 0.9130
Epoch 6/100
113/113 [==============================] - 36s 322ms/step - loss: 0.0149 - accuracy: 0.9988 - val_loss: 0.3053 - val_accuracy: 0.9116
Epoch 7/100
113/113 [==============================] - 36s 317ms/step - loss: 0.0134 - accuracy: 0.9988 - val_loss: 0.3083 -

In [ ]:
#Saving the model
mobilenet.save('/content/drive/MyDrive/iic-final/mnet-best-model.hdf5')

In [ ]:
#Evaluating the test data
mobilenet.evaluate_generator(test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.36309733986854553, 0.9126666784286499]

In [ ]:
#Scaling the inferance data
inferance_image_generator = ImageDataGenerator(rescale=1./255)
inferance_data = inferance_image_generator.flow_from_directory(inferance_data_dir,
                                       target_size=IMG_SIZE, batch_size=BATCH_SIZE,shuffle=False)

Found 3003 images belonging to 6 classes.


In [ ]:
#evalating the inferance data
mobilenet.evaluate_generator(inferance_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[1.618497371673584, 0.7565767765045166]